# Import Libraries

In [1]:
import requests
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from datetime import datetime
from io import StringIO
import sqlite3
# import icecream

In [2]:
# pip install db-sqlite3

In [3]:
# %pip install requests pandas numpy bs4 sqlite3 lxml

In [4]:
soup = BeautifulSoup(requests.get('https://web.archive.org/web/20230908091635/https://en.wikipedia.org/wiki/List_of_largest_banks').text,'html.parser')
table = soup.find('span',string='By market capitalization').find_next('table')
df_main = pd.read_html(StringIO(str(table)))[0]


In [5]:
df_main

,Rank,Bank name,Market cap (US$ billion)
0,1,JPMorgan Chase,432.92
1,2,Bank of America,231.52
2,3,Industrial and Commercial Bank of China,194.56
3,4,Agricultural Bank of China,160.68
4,5,HDFC Bank,157.91
5,6,Wells Fargo,155.87
6,7,HSBC Holdings PLC,148.90
7,8,Morgan Stanley,140.83
8,9,China Construction Bank,139.82
9,10,Bank of China,136.81


# Logs


In [6]:
def save_logs(message):
    with open('./logs/code_log.txt','a') as f:
        f.write(f'{datetime.now()}: {message}')

In [7]:
datetime.now()

datetime.datetime(2025, 6, 21, 19, 50, 32, 69566)

# Extract

In [8]:
def extract(url):
    soup = BeautifulSoup(requests.get(url).text,'html.parser')
    table = soup.find('span',string='By market capitalization').find_next('table')
    df = pd.read_html(StringIO(str(table)))[0]
    save_logs("Data has been extracted")
    return df


In [9]:
# C:\Users\Teacher\AppData\Local\Temp\ipykernel_7712\2908454608.py:6: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
#   df = pd.read_html(str(table))[0]


## Transform

In [10]:
def transform(df, csv_path):
    exchange_rate =  pd.read_csv(csv_path,index_col=0).to_dict()['Rate']
    print(exchange_rate)
    df['Market cap (Pkr billion)s'] = df['Market cap (US$ billion)'] *  exchange_rate['Pkr']
    df['Market cap (Inr billion)s'] = df['Market cap (US$ billion)'] * exchange_rate['Inr']
    df['Market cap (Eur billion)s'] = df['Market cap (US$ billion)'] * exchange_rate['Eur']
    df['Market cap (GBP billion)s'] = df['Market cap (US$ billion)'] * exchange_rate['GBP']
    # print(df)
    
    # save logs
    save_logs('Data Transformation Completed')

    return df


## Load

In [11]:
datebase_name = './Ouput/banks_db.db'
with sqlite3.connect(datebase_name) as conn:
    cursor = conn.cursor()
    result =cursor.execute("select 1")
    print(result.fetchall())



[(1,)]


In [17]:
def load_data_csv(df,output_csv_path):
    df.to_csv(output_csv_path)

    # save logs
    save_logs('Data saved to csv file.')

    return None

In [ ]:
# load to sqlite
def load_data_sql(df,table_name,conn):
    df.to_sql(table_name,conn,if_exists='replace',index=False)
    
def run_query(table_name,conn):
    cursor = conn.cursor()
    result =cursor.execute(('Select * from Largest_banks'))
    print(result.fetchall())

In [19]:
if __name__ == '__main__':
    url = 'https://web.archive.org/web/20230908091635/https://en.wikipedia.org/wiki/List_of_largest_banks'
    csv_path = './input/exchange_rate.csv'
    output_csv_path = './Ouput/transformed_exchange_rate.csv'    
    datebase_name = './Ouput/banks_db.db'
    table_name = 'Largest_banks'

    df = extract(url)

    df = transform(df,csv_path)
    load_data_csv(df,output_csv_path)
    # print(df)

    with sqlite3.connect(datebase_name) as conn:
        load_data_sql(df,table_name,conn)
        run_query(table_name,conn)
        

{'Pkr': 283.0, 'Inr': 86.0, 'Eur': 0.87, 'GBP': 0.74}
[(1,), (2,), (3,), (4,), (5,), (6,), (7,), (8,), (9,), (10,)]
